In [1]:
from pandas import DataFrame
from pandas imCaseAccelport Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from matplotlib import pyplot
import numpy as np
import pandas as pd
import tensorflow as tf
sess =tf.Session()

Using TensorFlow backend.


In [132]:
Data=pd.read_csv("~/Data/data_lstm.csv",header=None)
print(Data.shape)
Label=pd.read_csv("~/Data/label_lstm.csv",header=None)
print(Label.shape)


(21899, 11)
(21899, 1)


In [133]:
for i in range(Data.shape[1]):
    scaler=MinMaxScaler(feature_range=(-1,1))
    scaler=scaler.fit(Data.iloc[:,i].values.reshape(-1,1))
    Data.iloc[:,i]= scaler.transform(Data.iloc[:,i].values.reshape(-1,1))
    print("feature" ,i)
    print(scaler.data_max_)
    print(scaler.data_min_)
    print(scaler.data_range_)

feature 0
[5.98146725]
[-3.17332602]
[9.15479326]
feature 1
[17.00331688]
[-17.63447762]
[34.63779449]
feature 2
[1.33007812]
[-1.74121094]
[3.07128906]
feature 3
[1.99993896]
[-2.]
[3.99993896]
feature 4
[1.77966309]
[-1.45532227]
[3.23498535]
feature 5
[75.85366058]
[-119.63414764]
[195.48780823]
feature 6
[85.54878235]
[-86.34146881]
[171.89025116]
feature 7
[96.76829529]
[-134.63415527]
[231.40245056]
feature 8
[94.60174561]
[0.]
[94.60174561]
feature 9
[59.79333115]
[0.]
[59.79333115]
feature 10
[179.88510132]
[-179.64160156]
[359.52670288]


In [127]:
print(scaler.data_max_)
print(scaler.data_min_)
print(scaler.data_range_)

[179.88510132]
[-179.64160156]
[359.52670288]


In [151]:
Data.columns= ['CaseSpeed','CaseAccel','AccelX','AccelY','AccelZ','GyroX','GyroY','Gyroz','Roll','Pitch','Yaw']
Label.columns=['Push']

In [247]:
y_data=Label
#x_data=Data[['CaseSpeed','CaseAccel','Gyroz']]
x_data=Data[['CaseSpeed','CaseAccel','AccelX','AccelY','AccelZ','GyroX','GyroY','Gyroz','Roll','Pitch','Yaw']]
feature_size=x_data.shape[1]

#print(feature_size)
#print(x_data.shape)

In [248]:
seq_length = 50
x_array=[]
y_array=[]
for i in range(len(y_data)-seq_length):
    x_array.append(x_data.iloc[i:i+seq_length].get_values().reshape(1,-1))
    y_array.append(y_data.iloc[i+seq_length].get_values().reshape(1,-1))


In [249]:
print(np.array(np.squeeze(x_array)).shape)
print(np.array(np.squeeze(y_array)).shape)

(21849, 550)
(21849,)


In [250]:
x_vals=np.array(np.squeeze(x_array))
y_vals=np.array(np.squeeze(y_array))


In [251]:
all#print(len(x_vals))
train_indices = np.random.choice(len(x_vals),
                                 round(len(x_vals)*0.80),
                                 replace=False)
test_indices = np.array(list(set(range(len(x_vals))) - set(train_indices)))

x_vals_train = x_vals[train_indices]
x_vals_test = x_vals[test_indices]
y_vals_train = y_vals[train_indices]
y_vals_test = y_vals[test_indices]
#print(y_vals_train)
#print(len(y_vals_train))
#print(x_vals_train[3])

In [252]:
batch=500
new_len=int(x_vals_train.shape[0]/batch)*batch
x_vals_train=x_vals_train[:new_len]
y_vals_train=y_vals_train[:new_len]
#print(x_vals_train.shape)


In [253]:
new_len=int(y_vals_train.shape[0]/batch)*batch
x_vals_test=x_vals_test[:new_len]
y_vals_test=y_vals_test[:new_len]

In [254]:
def fit_lstm(X,y, batch_size, nb_epoch, neurons):
	X = X.reshape(X.shape[0], seq_length, feature_size)
	model = Sequential()
	model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1],X.shape[2]), stateful=True))
	model.add(Dense(1))
	model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])
	for i in range(nb_epoch):
		model.fit(X, y, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
		model.reset_states()
	return model


In [255]:
lstm_model=fit_lstm(x_vals_train,y_vals_train,batch,500,1)

Epoch 1/1
 - 2s - loss: 1.2198 - acc: 0.0000e+00
Epoch 1/1
 - 1s - loss: 1.0519 - acc: 0.0000e+00
Epoch 1/1
 - 1s - loss: 0.9400 - acc: 0.0000e+00
Epoch 1/1
 - 1s - loss: 0.8601 - acc: 0.0000e+00
Epoch 1/1
 - 1s - loss: 0.7951 - acc: 0.0000e+00
Epoch 1/1
 - 1s - loss: 0.7368 - acc: 5.2941e-04
Epoch 1/1
 - 1s - loss: 0.6836 - acc: 0.0202
Epoch 1/1
 - 1s - loss: 0.6401 - acc: 0.1228
Epoch 1/1
 - 1s - loss: 0.6122 - acc: 0.4402
Epoch 1/1
 - 1s - loss: 0.5971 - acc: 0.6084
Epoch 1/1
 - 1s - loss: 0.5860 - acc: 0.6280
Epoch 1/1
 - 1s - loss: 0.5739 - acc: 0.6086
Epoch 1/1
 - 1s - loss: 0.5594 - acc: 0.5884
Epoch 1/1
 - 1s - loss: 0.5422 - acc: 0.5771
Epoch 1/1
 - 1s - loss: 0.5256 - acc: 0.5740
Epoch 1/1
 - 1s - loss: 0.5116 - acc: 0.5779
Epoch 1/1
 - 1s - loss: 0.4977 - acc: 0.5889
Epoch 1/1
 - 1s - loss: 0.4827 - acc: 0.6024
Epoch 1/1
 - 1s - loss: 0.4666 - acc: 0.6165
Epoch 1/1
 - 1s - loss: 0.4492 - acc: 0.6308
Epoch 1/1
 - 1s - loss: 0.4299 - acc: 0.6429
Epoch 1/1
 - 1s - loss: 0.4104 

Epoch 1/1
 - 1s - loss: 0.2510 - acc: 0.8162
Epoch 1/1
 - 1s - loss: 0.2507 - acc: 0.8171
Epoch 1/1
 - 1s - loss: 0.2504 - acc: 0.8168
Epoch 1/1
 - 1s - loss: 0.2502 - acc: 0.8166
Epoch 1/1
 - 1s - loss: 0.2499 - acc: 0.8172
Epoch 1/1
 - 1s - loss: 0.2497 - acc: 0.8173
Epoch 1/1
 - 1s - loss: 0.2494 - acc: 0.8176
Epoch 1/1
 - 1s - loss: 0.2492 - acc: 0.8179
Epoch 1/1
 - 1s - loss: 0.2489 - acc: 0.8181
Epoch 1/1
 - 1s - loss: 0.2486 - acc: 0.8181
Epoch 1/1
 - 1s - loss: 0.2484 - acc: 0.8185
Epoch 1/1
 - 1s - loss: 0.2481 - acc: 0.8188
Epoch 1/1
 - 1s - loss: 0.2479 - acc: 0.8189
Epoch 1/1
 - 1s - loss: 0.2476 - acc: 0.8189
Epoch 1/1
 - 1s - loss: 0.2473 - acc: 0.8191
Epoch 1/1
 - 1s - loss: 0.2471 - acc: 0.8194
Epoch 1/1
 - 1s - loss: 0.2468 - acc: 0.8193
Epoch 1/1
 - 1s - loss: 0.2466 - acc: 0.8196
Epoch 1/1
 - 1s - loss: 0.2463 - acc: 0.8197
Epoch 1/1
 - 1s - loss: 0.2461 - acc: 0.8198
Epoch 1/1
 - 1s - loss: 0.2458 - acc: 0.8199
Epoch 1/1
 - 1s - loss: 0.2456 - acc: 0.8201
Epoch 1/1


 - 1s - loss: 0.2148 - acc: 0.8430
Epoch 1/1
 - 1s - loss: 0.2147 - acc: 0.8432
Epoch 1/1
 - 1s - loss: 0.2145 - acc: 0.8433
Epoch 1/1
 - 1s - loss: 0.2144 - acc: 0.8434
Epoch 1/1
 - 1s - loss: 0.2143 - acc: 0.8436
Epoch 1/1
 - 1s - loss: 0.2142 - acc: 0.8439
Epoch 1/1
 - 1s - loss: 0.2141 - acc: 0.8439
Epoch 1/1
 - 1s - loss: 0.2140 - acc: 0.8439
Epoch 1/1
 - 1s - loss: 0.2139 - acc: 0.8442
Epoch 1/1
 - 1s - loss: 0.2138 - acc: 0.8443
Epoch 1/1
 - 1s - loss: 0.2137 - acc: 0.8444
Epoch 1/1
 - 1s - loss: 0.2136 - acc: 0.8444
Epoch 1/1
 - 1s - loss: 0.2135 - acc: 0.8447
Epoch 1/1
 - 1s - loss: 0.2134 - acc: 0.8446
Epoch 1/1
 - 1s - loss: 0.2133 - acc: 0.8446
Epoch 1/1
 - 1s - loss: 0.2132 - acc: 0.8446
Epoch 1/1
 - 1s - loss: 0.2131 - acc: 0.8448
Epoch 1/1
 - 1s - loss: 0.2130 - acc: 0.8449
Epoch 1/1
 - 1s - loss: 0.2129 - acc: 0.8451
Epoch 1/1
 - 1s - loss: 0.2127 - acc: 0.8459
Epoch 1/1
 - 1s - loss: 0.2127 - acc: 0.8458
Epoch 1/1
 - 1s - loss: 0.2126 - acc: 0.8458
Epoch 1/1
 - 1s - lo

In [256]:
print(y_vals_test.shape[0])
print(x_vals_test.shape)
x_vals_test = x_vals_test[:4000]
y_vals_test = y_vals_test[:4000]
print(y_vals_test.shape[0])
print(x_vals_test.shape[0])

4370
(4370, 550)
4000
4000


In [257]:
#predictions=list()
#for i in range(0,y_vals_test.shape[0]):
x_vals_test_reshape=x_vals_test.reshape(x_vals_test.shape[0],seq_length,feature_size)
print(x_vals_test_reshape.shape)

yhat=lstm_model.predict(x_vals_test_reshape,batch)
#predictions.append(yhat)
predictions=yhat;



(4000, 50, 11)


In [258]:
print(yhat[0])

[-0.892122]


In [259]:
lstm_model.save_weights('LSTM_SPEED_ACCEL.h5')

In [260]:
print(x_vals_test_reshape[1])

[[ 0.40850434  0.04852006 -0.59674086  0.55385672 -0.04297951  0.06113536
   0.23944664  0.09723322  0.78661816  0.3917976   0.51083412]
 [ 0.41319547  0.04617078 -0.59674086  0.55385672 -0.04297951  0.06113536
   0.23944664  0.09723322  0.78661816  0.3917976   0.51083412]
 [ 0.41414104  0.05290029 -0.59674086  0.55385672 -0.04297951  0.06113536
   0.23944664  0.09723322  0.78661816  0.3917976   0.51083412]
 [ 0.40881519  0.05077289 -0.59674086  0.55385672 -0.04297951  0.06113536
   0.23944664  0.09723322  0.78661816  0.3917976   0.51083412]
 [ 0.41162497  0.03681375 -0.06915739  0.58919661 -0.1600317   0.30630068
   0.10606599  0.2047431   0.781173    0.39844153  0.51600051]
 [ 0.4091263   0.02639389 -0.06915739  0.58919661 -0.1600317   0.30630068
   0.10606599  0.2047431   0.781173    0.39844153  0.51600051]
 [ 0.40850434  0.02160961 -0.06915739  0.58919661 -0.1600317   0.30630068
   0.10606599  0.2047431   0.781173    0.39844153  0.51600051]
 [ 0.39749116  0.0150965  -0.06915739  0.

In [261]:
scores = lstm_model.evaluate(x_vals_test_reshape,y_vals_test,batch_size=batch)

4000/4000 [==============================] - 0s 105us/step


In [262]:
print(scores)

[0.21533903665840626, 0.8479999974370003]


In [263]:
print(len(predictions))
push_index_pre=list()
for i in range(len(predictions)):
    if(predictions[i]>0):
        predictions[i]=1
        push_index_pre.append(i)
    else:
        predictions[i]=-1


4000


In [264]:
#print(len(push_index_pre))
#print(push_index_pre)

In [265]:
predictions=np.squeeze(predictions)
print(predictions+1)

[0. 0. 0. ... 0. 0. 0.]


In [266]:
print(y_vals_test.shape)

(4000,)


In [267]:

accuracy = tf.reduce_mean(tf.cast(tf.equal(predictions, y_vals_test), tf.float32))
print(sess.run(accuracy))

0.93275


In [268]:
match=tf.equal(predictions, y_vals_test)
print(match.shape)

(4000,)


In [269]:
print(lstm_model.layers[0].weights)

[<tf.Variable 'lstm_13/kernel:0' shape=(11, 4) dtype=float32_ref>, <tf.Variable 'lstm_13/recurrent_kernel:0' shape=(1, 4) dtype=float32_ref>, <tf.Variable 'lstm_13/bias:0' shape=(4,) dtype=float32_ref>]


In [270]:
print(lstm_model.layers[0].get_weights())

[array([[-2.467588  , -0.4748822 , -0.29248804,  1.5438935 ],
       [ 6.879515  , -0.9053682 , -3.8640919 , -4.058645  ],
       [-3.0119777 , -0.16483419, -0.09317629,  0.8482545 ],
       [-0.50112706,  0.27462587,  0.20877554, -0.45988902],
       [-0.8812889 ,  0.7433692 , -0.04812019, -1.0335091 ],
       [ 0.62394816, -0.18716961,  0.05548005, -0.48061693],
       [ 2.8513088 ,  0.270967  ,  0.5809678 , -1.3313452 ],
       [ 1.8459439 ,  0.23768651,  0.11361132,  0.09316795],
       [ 0.3595688 ,  0.6126058 ,  0.34139833, -0.14293793],
       [ 0.76729995,  0.5086219 , -0.8453934 , -0.22615564],
       [-0.32726464, -0.08386017, -0.04319441,  0.1744854 ]],
      dtype=float32), array([[ 1.9057307 , -0.30936152,  0.53360456, -2.9849014 ]],
      dtype=float32), array([ 0.22584009,  1.2504009 ,  0.3241909 , -0.3518248 ], dtype=float32)]


In [271]:
print((lstm_model.layers[1].get_weights()))

[array([[-1.6313976]], dtype=float32), array([-0.6648892], dtype=float32)]


In [243]:
print((lstm_model.layers[1].activation))

<function linear at 0x7f9363cdb6a8>


In [244]:
print((lstm_model.weights))

[<tf.Variable 'lstm_12/kernel:0' shape=(3, 4) dtype=float32_ref>, <tf.Variable 'lstm_12/recurrent_kernel:0' shape=(1, 4) dtype=float32_ref>, <tf.Variable 'lstm_12/bias:0' shape=(4,) dtype=float32_ref>, <tf.Variable 'dense_12/kernel:0' shape=(1, 1) dtype=float32_ref>, <tf.Variable 'dense_12/bias:0' shape=(1,) dtype=float32_ref>]


In [245]:
print(lstm_model.get_weights())

[array([[-2.9086483 , -1.3354431 , -0.41930252,  1.1520714 ],
       [10.030381  ,  0.78706133, -6.2370963 , -3.346558  ],
       [-0.08319151,  0.01163469,  0.11354639,  0.80962485]],
      dtype=float32), array([[ 0.7697514 , -0.68766826,  0.44904068, -1.5737821 ]],
      dtype=float32), array([2.1102474 , 2.1775978 , 0.7336147 , 0.22738917], dtype=float32), array([[-1.0524479]], dtype=float32), array([-0.5111496], dtype=float32)]


In [246]:
print((lstm_model.weights))

[<tf.Variable 'lstm_12/kernel:0' shape=(3, 4) dtype=float32_ref>, <tf.Variable 'lstm_12/recurrent_kernel:0' shape=(1, 4) dtype=float32_ref>, <tf.Variable 'lstm_12/bias:0' shape=(4,) dtype=float32_ref>, <tf.Variable 'dense_12/kernel:0' shape=(1, 1) dtype=float32_ref>, <tf.Variable 'dense_12/bias:0' shape=(1,) dtype=float32_ref>]


In [182]:
lstm_model.save_weights("model.h5")

In [67]:
print(lstm_model.get_weights())

[array([[-2.0519213e-01,  4.5701525e-01,  5.6810164e-01, -6.5351076e-02,
         3.2100949e-01,  4.0525311e-01,  3.2218349e-01, -3.3198297e-01,
        -5.8441274e-02,  2.5772029e-01,  3.0936098e-01,  3.7396160e-01,
         1.2128439e-01,  4.9625224e-01, -6.1145779e-03, -1.6282354e-01,
         3.9555740e-02,  2.5088429e-01,  2.0375276e-01, -7.9840146e-02,
         1.6427815e-01,  1.7783567e-01,  1.9600633e-01,  3.5709006e-01,
        -4.3715473e-02,  2.7845672e-01, -9.3587600e-02, -1.1968474e-01,
        -2.2350639e-01,  2.2780266e-01, -2.4385601e-01,  1.2654191e-01,
        -2.2941810e-01,  3.6963925e-02,  2.3294114e-01, -7.4179471e-02,
        -9.4008744e-03,  2.4523064e-01,  3.8190812e-01, -4.7663590e-01,
         2.8878564e-01,  5.7949066e-02,  1.8276794e-01, -1.3898808e-02,
         1.6178323e-02,  1.8885452e-01,  2.4988143e-01, -2.3832142e-01,
        -2.9681221e-01,  1.0154809e-02, -3.3184800e-02,  5.6644864e-02,
         1.5588007e-03,  4.9233124e-02,  3.0181202e-01,  2.3018

In [42]:
print(lstm_model.weights)

[<tf.Variable 'lstm_2/kernel:0' shape=(11, 128) dtype=float32_ref>, <tf.Variable 'lstm_2/recurrent_kernel:0' shape=(32, 128) dtype=float32_ref>, <tf.Variable 'lstm_2/bias:0' shape=(128,) dtype=float32_ref>, <tf.Variable 'dense_2/kernel:0' shape=(32, 1) dtype=float32_ref>, <tf.Variable 'dense_2/bias:0' shape=(1,) dtype=float32_ref>]


In [169]:
print(sess.run(tf.equal((predictions), y_vals_test)))

[False False False ... False False False]


In [170]:
print(yhat)
print(yhat.shape)


[[-1.0255537 ]
 [-1.0128306 ]
 [-0.97264886]
 ...
 [-0.85350156]
 [-0.81048137]
 [-0.96145207]]
(4000, 1)
